# 1. Data Collection - PART 2
## Notebook to scrap each page 
We generate links based on the stockno that we grabed and stored into car_list.csv file. to get further details we click the 'see all specifications' button and scrap some more 

In [75]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import math
import os
chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

### 1. Opening our car_list.csv

In [76]:
prod_list=pd.read_csv('../data/scrape/product_links_coffee_table.csv')
prod_list.drop('Unnamed: 0',axis =1, inplace = True)
print(prod_list.shape)
prod_list.tail()                    

(3926, 2)


,link,type
3921,https://www.overstock.com/Home-Garden/Mercana-...,coffee table
3922,https://ad.tagdelivery.com/click?id=5a25d1c7-8...,coffee table
3923,https://ad.tagdelivery.com/click?id=485f4e72-9...,coffee table
3924,https://ad.tagdelivery.com/click?id=57d107ec-a...,coffee table
3925,https://ad.tagdelivery.com/click?id=2aed03c1-a...,coffee table


### 2. Setting up all the headings/parameters I need and an empty dataframe 

In [80]:
# list out all heading, parameters we will get from scraping a page. 
# add stock_no so that later on i can join the car_list.csv and car_details.csv  
heading1 =['page_link','product_name','item_no','description','style',\
          'material','type_class','color','rating']

#initialize an empty dataframe
df_product=pd.DataFrame(columns=heading1)
df_product

,page_link,product_name,item_no,description,style,material,type_class,color,rating


In [81]:
# list out all heading, parameters we will get from scraping a page. 
# add stock_no so that later on i can join the car_list.csv and car_details.csv  
heading2 =['image_link','product_name','page_link']

#initialize an empty dataframe
df_images=pd.DataFrame(columns=heading2)
df_images

,image_link,product_name,page_link


### 2. Scrap it

In [82]:
#counter to check where we are, track if we ge interrupted
counter=3001
# start nect at 3001

In [83]:
# condition for x_path
#row_selector = '//div[@class ="card--text-block-list--row"]'

driver = webdriver.Chrome(chromedriver)
#loop through the car_list we have made.  maually set start and finish. Use exit counter to know where you are at 
for link_str in prod_list.loc[3001:,'link']:
    
    print('----START PAGE ----')
    print(link_str)
    print('COUNT= ',counter)
    driver.get(str(link_str))
    time.sleep(2)
        
    page_link = link_str
    
    # get product name
    product_name_selector = '//h1'
    product_name = driver.find_element_by_xpath(product_name_selector).text
    #print(product_name)
    
    # get item_no
    item_no_selector ='//p[@class ="description-item-number"]'
    item_no=driver.find_element_by_xpath(item_no_selector).text.split()[1]
    item_no = int(item_no)
    #print(item_no)
    
    # get description
    description_selector ='//span[@itemprop ="description"]'
    description=driver.find_element_by_xpath(description_selector).text.split('\n')[0]
    #print(description)
    
    # get style , material, type_class, color
    style_selector = '//td[contains(text(),"Style")]/following-sibling::td'
    material_selector = '//td[contains(text(),"Material")]/following-sibling::td'
    type_class_selector = '//td[contains(text(),"Type")]/following-sibling::td'
    color_selector = '//td[contains(text(),"Color")]/following-sibling::td'
    
    try:
        style = driver.find_element_by_xpath(style_selector).text.strip()
    except:
        style = ""       
    try:
        material = driver.find_element_by_xpath(material_selector).text.strip()
    except:
        material = ""    
    try:
        type_class = driver.find_element_by_xpath(type_class_selector).text.strip()
    except:
        type_class = ""
    try:
        color = driver.find_element_by_xpath(color_selector).text.strip()
    except:
        color = ""

    #print(style,material,color,type_class)
    
    # get rating
    rating_selector ='//span[@class ="overall-rating"]'
    try:
        rating=driver.find_element_by_xpath(rating_selector).text
    except:
        rating = 0
    
    rating = float(rating)
    #print(rating)

    temp_list = [page_link,product_name,item_no,description,style,material,type_class,color,rating]
    try:
        # put into a  first panda
        df_product = df_product.append(pd.DataFrame([temp_list], columns=heading1),ignore_index=True)
    except:
        print('OPPS A ... no put in pandas')
    

    ###### images ##
    image_panel = '//div[@class="gallery-controls"]//div[@class ="thumb-frame"]//child::li'
    for row in driver.find_elements_by_xpath(image_panel):
        image_link = row.get_attribute('data-max-img')
        temp_img_list = [image_link,product_name,page_link]
        try:
        # put into a second panda
            df_images = df_images.append(pd.DataFrame([temp_img_list], columns=heading2),ignore_index=True)
        except:
            print('OPPS B ... no put in pandas')      
    
    #print(len(driver.find_elements_by_xpath(image_panel)))
    counter +=1
    print('----END PAGE-----')
    
driver.close()
  

----START PAGE ----
https://www.overstock.com/Home-Garden/Provincial-Home-Contemporary-Rustic-Palm-Island-Coffee-Table/21235108/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2825
COUNT=  3001
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/A.R.T.-Furniture-Epicenters-Williamsburg-Slat-Cocktail-Table/11845336/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2826
COUNT=  3002
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Chefs-Prep-Cart/11651609/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2827
COUNT=  3003
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Adalyn-Coffee-Table/9305711/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2828
COUNT=  3004
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Biony-Espresso-Wood-3-Piece-Coffee-Table-Set-with-Nail-Head-Trim/22965876/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2829
COUN

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Best-Quality-Furniture-Traditional-Cappuccino-Coffee-Table/17833582/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2866
COUNT=  3042
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modrest-Agate-Walnut-Coffee-Table/10461177/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2867
COUNT=  3043
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Safavieh-Couture-High-Line-Collection-Ninibel-Large-Veneer-Espresso-Coffee-Table/14143107/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2868
COUNT=  3044
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Salina-coffee-table/21377511/product.html?refccid=TVQWYCQRQ7KJ3ESHNKK7GJJKTI&searchidx=2869
COUNT=  3045
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Aurelle-Home-Italian-Modern-White-Coffee-Table/24121285/product.html?refccid=TVQWYCQRQ

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Contemporary-Black-Nickel-Coffee-Table/21339028/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2894
COUNT=  3074
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/White-Metal-Crystal-Shell-Table/17824979/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2895
COUNT=  3075
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Sleek-and-Stylish-Coffee-Table-Brown/18067665/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2896
COUNT=  3076
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Smoke-Grey-Oak-and-Black-Coffee-Table/19492314/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2897
COUNT=  3077
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Manhattan-Comfort-Ellis-Brown-and-Black-Metal-and-Wood-Coffee-Table/17490053/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Renwil-Blaine-Coffee-Table/21030539/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2934
COUNT=  3114
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Morgan-3-Piece-Beveled-Glass-Accent-Table-Set/9261847/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2935
COUNT=  3115
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furnitech-TANGO-CT/13005551/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2936
COUNT=  3116
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Best-Quality-Furniture-Glass-Top-2-Piece-Coffee-and-End-Table-Set/20347783/product.html?refccid=L6LSX5X2C5O3EH2J7KZN64FCBQ&searchidx=2937
COUNT=  3117
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Mercana-Blok-Rectangular-Wooden-Handmade-Coffee-Table/25418089/product.html?refccid=L6LSX5X2C5O3EH2J7K

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modern-Mirrored-Coffee-Table-With-Double-X-framed-Base-Nickel-Black/24255874/product.html?refccid=EVQKEF5VT35LVYSM56MCKM5XBM&searchidx=2963
COUNT=  3147
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-Ashmont-Collection-3-Piece-Coffee-and-End-Table-Set-with-Glass-Tops-and-Silver-Metal-Frames/22818959/product.html?refccid=EVQKEF5VT35LVYSM56MCKM5XBM&searchidx=2964
COUNT=  3148
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modern-Stainless-Steel-and-Wood-Coffee-Table/19559642/product.html?refccid=EVQKEF5VT35LVYSM56MCKM5XBM&searchidx=2965
COUNT=  3149
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Handmade-Parota-Wood-San-Pedrito-Mission-Coffee-Table-Mexico/7594991/product.html?refccid=EVQKEF5VT35LVYSM56MCKM5XBM&searchidx=2966
COUNT=  3150
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/CalceFerr

...

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Jameson-Rustic-Reclaimed-Oak-Lift-top-Coffee-Table/22648760/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3547
COUNT=  3771
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Emerald-Home-Cubix-Tobacco-Brown-Coffee-Table/20349877/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3548
COUNT=  3772
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Thalberg-Contemporary-Acrylic-Chrome-Coffee-Table/20300787/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3549
COUNT=  3773
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Armen-Living-Logan-Contemporary-Coffee-Table-with-Black-Iron-Finish-and-Grey-Wood-Top/21451205/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3550
COUNT=  3774
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Armen

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Contemporary-Glossy-White-Coffee-Table/21339089/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3584
COUNT=  3808
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/East-At-Mains-Elia-Coffee-Table/21543201/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3585
COUNT=  3809
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/East-At-Mains-Sara-Coffee-Table/21543195/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3586
COUNT=  3810
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Broyhill-Seabrooke-Rectangular-Cocktail-Table/21621491/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3587
COUNT=  3811
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Bosch-Brown-Cherry-Glass-Top-Coffee-Table/23136323/product.html?refccid=ZEYDX3H6WRNLLSCBKA4O3RYTHA&searchidx=3

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-Contemporary-Ringwood-Cherry-Wood-Grain-Finish-Coffee-Table-with-Black-Metal-Frame/24982468/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3611
COUNT=  3839
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-Roslindale-Contemporary-Rustic-Wood-Grain-Finish-Coffee-Table-with-Silver-Metal-Legs/24982398/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3612
COUNT=  3840
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modrest-Claw-Modern-Small-Coffee-Table/25359890/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3613
COUNT=  3841
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modrest-Claw-Modern-Large-Coffee-Table/25359145/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3614
COUNT=  3842
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Modrest-Billy-Modern-B

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-Contemporary-Geometric-Glass-Nesting-Coffee-Tables-in-Painted-Metal-Finish-Black/25566363/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3648
COUNT=  3876
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-40-Wood-Storage-Coffee-Table-with-Wicker-Totes-Black/25566364/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3649
COUNT=  3877
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-36-Durable-Laminate-Top-Coffee-Table-with-Painted-Metal-X-Base-Marble-Gold/25566369/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3650
COUNT=  3878
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-42-Sleek-Contemporary-Mixed-Material-Coffee-Table-Marble/25566352/product.html?refccid=G5PDM4ZXNE6K3MCAGTHKNZBOTM&searchidx=3651
COUNT=  3879
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Ho

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Mercana-Pinera-I-Box-A-B-Coffee-Table/25628312/product.html?refccid=W2QEATVEFVDJ2S2NFH7DQHGJVU&searchidx=3673
COUNT=  3905
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Dumbo-Grey-Concrete-Coffee-Table/25601192/product.html?refccid=W2QEATVEFVDJ2S2NFH7DQHGJVU&searchidx=3674
COUNT=  3906
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Offex-31.5-Contemporary-Round-Coffee-Table-in-Concrete-Laminate-Finish-Gray/25604244/product.html?refccid=W2QEATVEFVDJ2S2NFH7DQHGJVU&searchidx=3675
COUNT=  3907
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Harper-Blvd-Saffron-Industrial-Adjustable-Height-Coffee-Table/25612694/product.html?refccid=W2QEATVEFVDJ2S2NFH7DQHGJVU&searchidx=3676
COUNT=  3908
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/HomeRoots-Furniture-Modern-Dark-Grey-Fiber-Reinforced-Natural-Concrete-

In [84]:
# what have I got ?
print(counter)
print(df_product.shape)
df_product.tail()

3926
(925, 9)


,page_link,product_name,item_no,description,style,material,type_class,color,rating
920,https://www.overstock.com/Home-Garden/Mercana-...,Mercana Kramer II Coffee Table,30937148,Using a combination of woven flat iron formed ...,"Industrial, Modern & Contemporary",Iron,Coffee Tables,Silver,0.0
921,https://ad.tagdelivery.com/click?id=5a25d1c7-8...,Essex Coffee Table,22958669,Essex Coffee Table is beautifully designed wit...,"Industrial, Rustic","Metal, Veneer",Coffee Tables,Grey,4.3
922,https://ad.tagdelivery.com/click?id=485f4e72-9...,Safavieh Alec Weathered Barnwood Coffee Table,15932344,Mix the transitional Alec coffee table with ot...,"Bohemian & Eclectic, Industrial","Metal, Wood",Coffee Tables,Brown,3.8
923,https://ad.tagdelivery.com/click?id=57d107ec-a...,Circular Lightly Beveled Mid-century Style Cof...,14647871,This circular coffee table showcases modern de...,Mid-Century Modern,Wood,Coffee Tables,Black,4.7
924,https://ad.tagdelivery.com/click?id=2aed03c1-a...,Prism Rectangular Glass Coffee Table,15831792,Prism rectangular coffee table is topped with ...,Modern & Contemporary,"Glass, Steel",Coffee Tables,Silver,4.4


In [85]:
df_product['page_link'].nunique()

869

In [86]:
print(df_images['page_link'].nunique())

869


In [87]:
df_images.tail()

,image_link,product_name,page_link
3113,https://ak1.ostkcdn.com/images/products/715675...,Circular Lightly Beveled Mid-century Style Cof...,https://ad.tagdelivery.com/click?id=57d107ec-a...
3114,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
3115,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
3116,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
3117,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...


### 3. Write my data frame to file
If the file does not exist then i write the dataframe with headers else if the file is already there then don't append with the header row 

In [88]:
file_name1 = '../data/scrape/product_details_coffee_table.csv'
from pathlib import Path
config = Path(file_name1)
if config.is_file():
    print('exist = yes')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=True)
    

exist = yes


In [89]:
file_name2 = '../data/scrape/image_data_coffee_table.csv'
from pathlib import Path
config = Path(file_name2)
if config.is_file():
    print('exist = yes')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=True)
    

exist = yes


### 4. Check it
read the file and check if stuff was written 

In [90]:
df_check1 = pd.read_csv(file_name1)
print(df_check1.shape)
df_check1.tail()

(3926, 10)


,Unnamed: 0,page_link,product_name,item_no,description,style,material,type_class,color,rating
3921,920,https://www.overstock.com/Home-Garden/Mercana-...,Mercana Kramer II Coffee Table,30937148,Using a combination of woven flat iron formed ...,"Industrial, Modern & Contemporary",Iron,Coffee Tables,Silver,0.0
3922,921,https://ad.tagdelivery.com/click?id=5a25d1c7-8...,Essex Coffee Table,22958669,Essex Coffee Table is beautifully designed wit...,"Industrial, Rustic","Metal, Veneer",Coffee Tables,Grey,4.3
3923,922,https://ad.tagdelivery.com/click?id=485f4e72-9...,Safavieh Alec Weathered Barnwood Coffee Table,15932344,Mix the transitional Alec coffee table with ot...,"Bohemian & Eclectic, Industrial","Metal, Wood",Coffee Tables,Brown,3.8
3924,923,https://ad.tagdelivery.com/click?id=57d107ec-a...,Circular Lightly Beveled Mid-century Style Cof...,14647871,This circular coffee table showcases modern de...,Mid-Century Modern,Wood,Coffee Tables,Black,4.7
3925,924,https://ad.tagdelivery.com/click?id=2aed03c1-a...,Prism Rectangular Glass Coffee Table,15831792,Prism rectangular coffee table is topped with ...,Modern & Contemporary,"Glass, Steel",Coffee Tables,Silver,4.4


In [91]:
df_check2 = pd.read_csv(file_name2)
print(df_check2.shape)
print(df_check2['page_link'].nunique())
df_check2.tail()

(18144, 4)
3694


,Unnamed: 0,image_link,product_name,page_link
18139,3113,https://ak1.ostkcdn.com/images/products/715675...,Circular Lightly Beveled Mid-century Style Cof...,https://ad.tagdelivery.com/click?id=57d107ec-a...
18140,3114,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
18141,3115,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
18142,3116,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...
18143,3117,https://ak1.ostkcdn.com/images/products/855432...,Prism Rectangular Glass Coffee Table,https://ad.tagdelivery.com/click?id=2aed03c1-a...


### next -> clean_data.ipynb 